In [1]:
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np
import gc
from tensorflow import keras
from keras import layers
from keras.models import Model
from bulid_transformer_model import bulid_model
from data_loading import load_data
from sequence_generator import seq_generator

2023-10-29 19:28:32.814839: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 19:28:32.854572: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-29 19:28:32.855551: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-29 19:28:33.667501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
(train_movie,train_trace,train_speed),_ = load_data(501271265)

/workspaces/mouse_visual_cortex_NN_modeling/data_loading.py:24: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  running_speed_train[np.where(running_speed_train=='Nan')] = 0
/workspaces/mouse_visual_cortex_NN_modeling/data_loading.py:42: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  running_speed_val[np.where(running_speed_train=='Nan')] = 0


In [3]:
model = bulid_model(num_heads=4,spatial_layers=3,temporal_layers=2,delay=40,embed_dim=128,output_shape=8)

data = seq_generator(x = train_movie,y = train_trace,r = train_speed,delay = 40,shuffle=True,batch_size=10)


In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import json
          
filter_json = """
[
    {
        "field": "reliability_nm3",
        "op": "between",
        "value": [
            0.5,
            1
        ]
    },
    {
        "field": "area",
        "op": "in",
        "value": [
            "VISl"
        ]
    }
]
"""
       
filters = json.loads(filter_json)
         
boc = BrainObservatoryCache(manifest_file="brain_observatory/manifest.json")
cells = boc.get_cell_specimens(filters=filters)

UndefinedVariableError: name 'reliability_nm3' is not defined

In [7]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
boc = BrainObservatoryCache(cache= True,manifest_file='../brain_data/brain_observatory_manifest.json')
depth = boc.get_all_imaging_depths()
all_areas = boc.get_all_targeted_structures()
experiments = boc.get_ophys_experiments(targeted_structures=['VISl'],imaging_depths=depth[:5],session_types=['three_session_A'],require_eye_tracking=True,cells)

In [11]:
for e in experiments:
    print(e['session_type'])

three_session_B
three_session_A
three_session_A
three_session_A
three_session_C
three_session_C2
three_session_A
three_session_C2
OPHYS_4_images_B
three_session_B
OPHYS_1_images_A
three_session_C2
three_session_A
three_session_C
three_session_B
three_session_B
three_session_B
three_session_B
OPHYS_1_images_A
three_session_C2
three_session_A
three_session_C2
OPHYS_2_images_A_passive
three_session_C2
OPHYS_5_images_B_passive
three_session_A
three_session_B
OPHYS_2_images_A_passive
three_session_B
three_session_C2
three_session_B
three_session_A
three_session_B
three_session_A
three_session_C2
three_session_B
three_session_A
three_session_B
three_session_B
three_session_B
three_session_A
three_session_B
three_session_B
OPHYS_5_images_B_passive
three_session_B
OPHYS_2_images_A_passive
three_session_A
OPHYS_5_images_B_passive
three_session_A
three_session_A
three_session_C
three_session_A
three_session_B
three_session_C2
OPHYS_1_images_A
three_session_C2
three_session_A
three_session_A
thre

In [12]:
boc.get_all_stimuli()

['drifting_gratings',
 'locally_sparse_noise',
 'locally_sparse_noise_4deg',
 'locally_sparse_noise_8deg',
 'natural_movie_one',
 'natural_movie_three',
 'natural_movie_two',
 'natural_scenes',
 'spontaneous',
 'static_gratings']

In [5]:
boc.get cotainers[0]['id']

564425775

In [4]:
from data_loading import load_data
train,val = load_data(experiment_id=501271265)

KeyboardInterrupt: 